In [1]:
import os
from glob import glob
from os.path import join, dirname
import nibabel as nib
import numpy as np

In [2]:
subj = 'SMC10'

In [3]:
work_dir = '/Users/clmn/Desktop/Samsung_Hospital'
fwhm = 4
thresh_motion = 0.4

In [4]:
raw_dir = join(work_dir,subj,'raw')
epi_dir = glob(join(raw_dir,'SMC_*_fMRI'))[0]
T1_dir = glob(join(raw_dir,'SMC_*_T1'))[0]

In [5]:
print(epi_dir,'\n',T1_dir)

/Users/clmn/Desktop/Samsung_Hospital/SMC10/raw/SMC_2020_10_200617_fMRI 
 /Users/clmn/Desktop/Samsung_Hospital/SMC10/raw/SMC_2020_10_200617_T1


In [6]:
print('dcm2niix_afni -o %s %s/*.dcm' %(raw_dir,epi_dir))

dcm2niix_afni -o /Users/clmn/Desktop/Samsung_Hospital/SMC10/raw /Users/clmn/Desktop/Samsung_Hospital/SMC10/raw/SMC_2020_10_200617_fMRI/*.dcm


In [7]:
epi = glob(join(raw_dir,'SMC*_fMRI.nii.gz'))[0]
t1 = glob(join(raw_dir,'SMC*_T1.nii.gz'))[0]

In [8]:
output_dir = join(work_dir,'preproc_data',subj)
os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)
os.makedirs('./preprocessed', exist_ok=True)

## S.S. Kim's code

In [9]:
os.chdir('./preprocessed')

## tcat
os.system('3dTcat -tr 3 -prefix pb00.%s.rest.tcat %s' %(subj,epi))
os.system('3dcopy %s ./%s.anat+orig' %(t1,subj))
os.system('3dToutcount -automask -fraction -polort 3 -legendre')

256

In [10]:
#================================ despike =================================
## Removes 'spikes' from the 3D+time input dataset and writes a new dataset with the spike values 
## replaced by something more pleasing to the eye.
os.system('3dDespike -NEW -nomask -prefix pb00.%s.rest.despike pb00.%s.rest.tcat+orig' %(subj,subj))

0

In [11]:
# ================================= tshift =================================
## t shift or slice time correction
## time shift data so all slice timing is the same
## 데이터를 얻는(slicing) 시간이 각각의 axial voxel에 대해 다르기 때문에 보정해주는 것.
os.system('3dTshift -tzero 0 -quintic -prefix pb01.%s.rest.tshift pb00.%s.rest.despike+orig' %(subj,subj))
## tzero -> to interpolate all the slices as though they were all acquired at the beginning of each TR.
## quintic -> 5th order of polynomial

0

In [12]:
# ================================= align ==================================
## for e2a: compute anat alignment transformation to EPI registration base
## (new anat will be intermediate, stripped, epi_$subjID.anat_ns+orig)
## Here, the 'warp' is a 3x4 matrix = affine transform of space which the user supplies.
os.system('3dWarp -deoblique -prefix %s.anat.deoblique %s.anat+orig' %(subj,subj))
## skull strip
os.system('3dSkullStrip -input %s.anat.deoblique+orig -prefix %s.sSanat -orig_vol' %(subj,subj))
## The output dataset has the white matter (WM) intensity approximately uniformized 
## across space, and scaled to peak at about 1000.
os.system('3dUnifize -input %s.sSanat+orig -prefix %s.UnisSanat -GM' %(subj,subj))

0

In [13]:
# - align EPI to anatomical datasets or vice versa
os.system(
    'align_epi_anat.py -anat2epi -anat %s.UnisSanat+orig -anat_has_skull no \
    -epi pb01.%s.rest.tshift+orig   -epi_base 3 \
    -epi_strip 3dAutomask                                      \
    -suffix _al_junk                     -check_flip           \
    -volreg off    -tshift off           -ginormous_move       \
    -cost nmi      -align_centers yes' %(subj,subj)
)
# -cost lpa (local pearson correlation: nonlinear average of Pearson cc over local neighborhoods)
# -cost nmi (1/Normalized MI = H(base,source)/[H(base)+H(source)])

0

In [ ]:
# ================================= tlrc ==================================
# warp anatomy to standard space
os.system(
    '@auto_tlrc -base ~/abin/MNI152_T1_2009c+tlrc.HEAD -input %s.UnisSanat+orig \
    -no_ss -init_xform AUTO_CENTER' %subj
)
os.system('cat_matvec %s.UnisSanat+tlrc::WARP_DATA -I > warp.anat.Xat.1D' %subj)

In [ ]:
# ================================== register and warp ========================
# register each volume to the base
os.system(
    "3dvolreg -verbose -zpad 1 -cubic -base pb01.%s.rest.tshift+orig'[3]' \
    -1Dfile dfile.%s.rest.1D -prefix rm.epi.volreg.%s.rest \
    -1Dmatrix_save mat.rest.vr.aff12.1D \
    pb01.%s.rest.tshift+orig" %(subj,subj,subj,subj)
)

In [ ]:
# create an all-1 dataset to mask the extents of the warp
os.system('3dcalc -overwrite -a pb01.%s.rest.tshift+orig -expr 1 -prefix rm.%s.epi.all1' %(subj,subj))

# catenate volreg, epi2anat and tlrc transformations
os.system(
    'cat_matvec -ONELINE \
    %s.UnisSanat_al_junk_mat.aff12.1D -I \
    mat.rest.vr.aff12.1D > mat.%s.rest.warp.aff12.1D' %(subj,subj)
)

In [ ]:
# apply catenated xform : volreg, epi2anat and tlrc
os.system(
    '3dAllineate -base %s.UnisSanat+orig \
     -input pb01.%s.rest.tshift+orig \
     -1Dmatrix_apply mat.%s.rest.warp.aff12.1D \
     -master pb01.%s.rest.tshift+orig  -prefix rm.epi.nomask.%s.rest' %(subj,subj,subj,subj,subj)
)

In [28]:
# warp the all-1 dataset for extents masking
os.system(
    '3dAllineate -base %s.UnisSanat+orig \
     -input rm.%s.epi.all1+orig \
     -1Dmatrix_apply mat.%s.rest.warp.aff12.1D \
     -final NN -quiet \
     -master pb01.%s.rest.tshift+orig -prefix rm.epi.1.%s.rest' %(subj,subj,subj,subj,subj)
)

0

In [27]:
# make an extents intersection mask of this run
os.system('3dTstat -min -prefix rm.epi.min.%s.rest rm.epi.1.%s.rest+orig' %(subj,subj))
os.system('3dcopy rm.epi.min.%s.rest+orig mask_epi_extents.%s' %(subj,subj))
os.system("3dcalc -a rm.epi.nomask.%s.rest+orig -b mask_epi_extents.%s+orig \
-expr 'a*b' -prefix pb02.%s.rest.volreg" %(subj,subj,subj))

0

## modified

In [9]:
os.chdir('./preprocessed')

In [10]:
########
# ANAT #
########
# -------------------------- root 1 --------------------------
os.system('3dcopy %s %s.anat+orig.nii.gz' %(t1,subj))
## output : subj.anat+orig
os.system('3dWarp -deoblique -prefix %s.anat.deoblique %s.anat+orig' %(subj,subj))
## output : subj.anat.deoblique+orig

# -------------------------- branch 1-1 (Kim) --------------------------
os.system('3dSkullStrip -input %s.anat.deoblique+orig -prefix %s.anat.ss -orig_vol' %(subj,subj))
## output : subj.anat.ss+orig
os.system('3dUnifize -input %s.anat.ss+orig -prefix %s.anat.Unifize -GM' %(subj,subj))
## output : subj.anat.Unifize+orig

# -------------------------- branch 1-2 (Byeon) --------------------------
# os.system('3dresample -orient RAI -prefix %s.anat.rai -inset %s.anat.deoblique+orig' %(subj,subj))
# os.system('3dUnifize -input %s.anat.rai+orig -prefix %s.anat.unifize -GM -clfrac 0.5' %(subj,subj))
# os.system('3dSkullStrip -input %s.anat.unifize+orig -prefix %s.anat.ss -orig_vol' %(subj,subj))

# ================================= tlrc coordinate ==================================
# warp anatomy to standard space
os.system(
    '@auto_tlrc -base ~/abin/MNI152_T1_2009c+tlrc.HEAD -input %s.anat.Unifize+orig \
    -no_ss -init_xform AUTO_CENTER' %subj)
## output : 
## subj.anat.Unifize_shift.1D
## pre.subj.anat.Unifize+orig
## subj.anat.Unifize.Xaff12.1D
## subj.anat.Unifize.Xat.1D
## subj.anat.Unifize.maskwarp.Xat.1D
## subj.anat.Unifize+tlrc : 'whereami'

os.system('cat_matvec %s.anat.Unifize+tlrc::WARP_DATA -I > warp.anat.Xat.1D' %subj)
## output : warp.anat.Xat.1D

0

In [11]:
########
# REST #
########
#================================ tcat =================================
os.system('3dTcat -tr 3 -prefix pb00.%s.rest.tcat %s' %(subj,epi))
## output : pb00.subj.rest.tcat+orig

os.system('3dToutcount -automask -fraction -polort 3 -legendre pb00.%s.rest.tcat+orig > outcount.%s.1D' %(subj,subj))
# polort = the polynomial order of the baseline model
## output : outcount.subj.1D

# if ( `1deval -a outcount.$subj.r$run.1D"{0}" -expr "step(a-0.4)"` ) then
#     echo "** TR #0 outliers: possible pre-steady state TRs in run ${run}" >> out.pre_ss_warn.txt
# endif

#================================ despike =================================
## Removes 'spikes' from the 3D+time input dataset and writes a new dataset with the spike values 
## replaced by something more pleasing to the eye.
os.system('3dDespike -NEW -nomask -prefix pb00.%s.rest.despike pb00.%s.rest.tcat+orig' %(subj,subj))
## output : pb00.subj.rest.despike+orig

# ================================= tshift =================================
## t shift or slice time correction
## time shift data so all slice timing is the same
## 데이터를 얻는(slicing) 시간이 각각의 axial voxel에 대해 다르기 때문에 보정해주는 것.
os.system('3dTshift -tzero 0 -quintic -prefix pb01.%s.rest.tshift pb00.%s.rest.despike+orig' %(subj,subj))
## tzero -> to interpolate all the slices as though they were all acquired at the beginning of each TR.
## quintic -> 5th order of polynomial
## output : pb01.subj.rest.tshift+orig

0

In [ ]:
# -------------------------- branch 2-1 (anat to epi) --------------------------
# ================================= align ==================================
# - align EPI to anatomical datasets or vice versa
os.system(
    'align_epi_anat.py -anat2epi -anat %s.anat.Unifize+orig -anat_has_skull no \
    -epi pb01.%s.rest.tshift+orig   -epi_base 3 \
    -epi_strip 3dAutomask                                      \
    -suffix _al_junk                     -check_flip           \
    -volreg off    -tshift off           -ginormous_move       \
    -cost nmi      -align_centers yes' %(subj,subj))
# -cost lpa (local pearson correlation: nonlinear average of Pearson cc over local neighborhoods)
# -cost nmi (1/Normalized MI = H(base,source)/[H(base)+H(source)])
## output : 
# subj.anat.Unifize_unflipped+orig
# subj.anat.Unifize_unflipped_ob+orig
# subj.anat.Unifize_unflipped_ob_shft_I.1D
# __tt_pb01.subj.rest.tshift_ts+orig
# pb01.subj.rest.tshift_ts_ns+orig
# pb01.subj.rest.tshift_ts_ns_wt+orig
# subj.anat.Unifize_unflipped_ob_al_junk_wtal+orig
# subj.anat.Unifize_unflipped_ob_temp_al_junk+orig
# subj.anat.Unifize_al_junk_e2a_only_mat.aff12.1D
# __tt_subj.anat.Unifize_lr_al_junk+orig
# subj.anat.Unifize_flip_al_junk+orig
# subj.anat.Unifize_flip__al_junk_mat.aff12.1D
# __tt_lr_noflipcosts.1D
# __tt_lr_flipcosts.1D
# aea_checkflip_results.txt
# subj.anat.Unifize_al_junk_mat.aff12.1D *
# subj.anat.Unifize_al_junk+orig

# ================================== register and warp ========================
# register each volume to the base
os.system(
    "3dvolreg -verbose -zpad 1 -cubic -base pb01.%s.rest.tshift+orig'[3]' \
    -1Dfile dfile.%s.rest.1D -prefix rm.epi.volreg.%s.rest \
    -1Dmatrix_save mat.rest.vr.aff12.1D \
    pb01.%s.rest.tshift+orig" %(subj,subj,subj,subj))
## output : 
## mat.rest.vr.aff12.1D *
## rm.epi.volreg.subj.rest+orig
## dfile.subj.rest.1D

# create an all-1 dataset to mask the extents of the warp
os.system('3dcalc -overwrite -a pb01.%s.rest.tshift+orig -expr 1 -prefix rm.%s.epi.all1' %(subj,subj))
## output : rm.subj.epi.all1+orig

# catenate volreg, epi2anat and tlrc transformations
os.system(
    'cat_matvec -ONELINE \
    %s.anat.Unifize_al_junk_mat.aff12.1D -I \
    mat.rest.vr.aff12.1D > mat.%s.rest.warp.aff12.1D' %(subj,subj))
## output : mat.subj.rest.warp.aff12.1D

In [ ]:
# -------------------------- branch 2-2 (epi to anat) --------------------------
# ================================= align ==================================
# - align EPI to anatomical datasets or vice versa
os.system(
    'align_epi_anat.py -epi2anat -anat %s.anat.Unifize+orig -anat_has_skull no \
    -epi pb01.%s.rest.tshift+orig   -epi_base 3 \
    -epi_strip 3dAutomask                                      \
    -suffix _al_junk                     -check_flip           \
    -volreg off    -tshift off           -ginormous_move       \
    -cost nmi      -align_centers yes' %(subj,subj))
## output : 
# SMC10.anat.Unifize_unflipped+orig
# SMC10.anat.Unifize_unflipped_ob+orig
# SMC10.anat.Unifize_unflipped_ob_shft.1D
# SMC10.anat.Unifize_unflipped_shft_I.1D
# pb01.SMC10.rest.tshift_ts_ns+orig
# pb01.SMC10.rest.tshift_ts_ns_wt+orig
# SMC10.anat.Unifize_unflipped_ob_al_junk_wtal+orig
# SMC10.anat.Unifize_unflipped_ob_temp_al_junk+orig
# SMC10.anat.Unifize_al_junk_e2a_only_mat.aff12.1D
# SMC10.anat.Unifize_flip_al_junk+orig
# SMC10.anat.Unifize_flip__al_junk_mat.aff12.1D
# __tt_lr_noflipcosts.1D
# __tt_lr_flipcosts.1D
# aea_checkflip_results.txt
# pb01.SMC10.rest.tshift_al_junk_mat.aff12.1D
# pb01.SMC10.rest.tshift_al_junk+orig : subj.anat.Unifize+tlrc 와 align

# ================================== register and warp ========================
# register each volume to the base
os.system(
    "3dvolreg -verbose -zpad 1 -cubic -base pb01.%s.rest.tshift_al_junk+orig'[3]' \
    -1Dfile dfile.%s.rest.1D -prefix rm.epi.volreg.%s.rest \
    -1Dmatrix_save mat.rest.vr.aff12.1D \
    pb01.%s.rest.tshift_al_junk+orig" %(subj,subj,subj,subj))
## output : 
## mat.rest.vr.aff12.1D
## rm.epi.volreg.subj.rest+orig
## dfile.subj.rest.1D

# create an all-1 dataset to mask the extents of the warp
os.system("3dcalc -overwrite -a pb01.%s.rest.tshift_al_junk+orig -expr 'bool(a)' -prefix rm.%s.epi.all1" %(subj,subj))
## output : rm.subj.epi.all1+orig

In [30]:
os.system("3dcalc -a pb01.%s.rest.tshift_al_junk+orig -b rm.%s.epi.all1+orig \
    -expr 'a*b' -prefix pb02.%s.rest.volreg" %(subj,subj,subj))

0